# Assignment 1

This assignment will involve the creation of a spellchecking system and an evaluation of its performance. You may use the code snippets provided in Python for completing this or you may use the programming language or environment of your choice

Please start by downloading the corpus `holbrook.txt` from Blackboard

The file consists of lines of text, with one sentence per line. Errors in the line are marked with a `|` as follows

    My siter|sister go|goes to Tonbury .
    
In this case the word 'siter' was corrected to 'sister' and the word 'go' was corrected to 'goes'.

In some places in the corpus two words maybe corrected to a single word or one word to a multiple words. This is denoted in the data using underscores e.g.,

    My Mum goes out some_times|sometimes .
    
For the purpose of this assignment you do not need to separate these words, but instead you may treat them like a single token.

*Note: you may use any functions from NLTK to complete the assignment. It should not be necessary to use other libraries and so please consult with us if your solution involves any other external library. If you use any function from NLTK in Task 6 please include a brief description of this function and how it contributes to your solution.*

## Task 1 (10 Marks)

Write a parser that can read all the lines of the file `holbrook.txt` and print out for each line the original (misspelled) text, the corrected text and the indexes of any changes. The indexes refers to the index of the words in the sentence. In the example given, there is only an error in the 10th word and so the list of indexes is [9]. It is not necessary to analyze where the error occurs inside the word.

Then split your data into a test set of 100 lines and a training set.

In [67]:
from collections import defaultdict
import nltk
from nltk.metrics.distance import edit_distance
from nltk.util import ngrams

In [68]:
# Create parser function
def parse(text):

    # Create empty lists
    line_original = []
    line_corrected = []
    Index = []
    count = 0

    # create a for loop that goes through each line in the text
    for line in text:
        if '|' in line:
            # If sentence has '|', then split
            string = line.split('|')
            # Word before '|' store in line_original
            line_original.append(string[0])
            # Word after '|' store in line_original
            line_corrected.append(string[1])
            # Append count into Index where error occured
            Index.append(count)

        else:
            # If sentence does not include '|', then sentence is stored in both original & corrected as it is.
            line_original.append(line)
            line_corrected.append(line)
        count = count + 1

    #Loading to original, correct and index list to a dictionary
    dictionary = {'original' : line_original, 'corrected' : line_corrected, 'indexes' : Index}

    return dictionary

# Create preprocess_data function
def preprocess_data():
    # Initialize data
    data = []

    # Reading the txt file
    txt_file = open("holbrook.txt", "r")
    # Initialize lines
    lines = []

    for x in txt_file:
        lines.append(x.strip())

    # Using nltk package to tokenize sentences
    sentences = [nltk.word_tokenize(sentence) for sentence in lines]


    # Now implementing parser() function,
    # which was created to generate corrected & original sentences & index
    for sentence in sentences:
        data.append(parse(sentence))

    return data


# Calling postprocessing function
processed_data = preprocess_data()

# Test parser
print(processed_data[2])


{'original': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'siter', '.'], 'corrected': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'sister', '.'], 'indexes': [9]}


The counts and assertions given in the following sections are based on splitting the training and test set as follows

In [69]:
test = processed_data[:100]
train = processed_data[100:]
words_correct = [index['corrected'] for index in train]

## **Task 2** (10 Marks): 
Calculate the frequency (number of occurrences), *ignoring case*, of all words and their unigram probability from the corrected *training* sentences.

*Hint: use `Counter` to implement this so it may be called many times*

In [70]:
from collections import Counter

def build_unigram_model(words_correct):
    # initialize empty list 
    text = []
    for word in words_correct:
        # convert the list of strings to a string with a space between each
        text.append(" ".join(word).lower())

    # same as above
    text = " ".join(text)
    text = nltk.word_tokenize(text)

    # Use FreqDist to calculate text's frequency distribution.
    unig_freq = nltk.FreqDist(text)

    return unig_freq

def prob(word, unigram_model):
    total = unigram_model.N()
    freq, _ = get_word_stats(word, unigram_model)
    return freq / float(total)

unigram_model = build_unigram_model(words_correct)

count, token_num = get_word_stats('me', unigram_model)


# Test your code with the following
assert(count == 87)
# Another way to test code
# print(count)
print(prob('me', unigram_model) > prob('test', unigram_model))

True


## **Task 3** (15 Marks): 
[Edit distance](https://en.wikipedia.org/wiki/Edit_distance) is a method that calculates how similar two strings are to one another by counting the minimum number of operations required to transform one string into the other. There is a built-in implementation in NLTK that works as follows:


In [71]:
from nltk.metrics.distance import edit_distance


# Edit distance returns the number of changes to transform one word to another
print(edit_distance("hello", "hi"))

4


Write a function that calculates all words with *minimal* edit distance to the misspelled word. You should do this as follows

1. Collect the set of all unique tokens in `train`
2. Find the minimal edit distance, that is the lowest value for the function `edit_distance` between `token` and a word in `train`
3. Output all unique words in `train` that have this same (minimal) `edit_distance` value

*Do not implement edit distance, use the built-in NLTK function `edit_distance`*

In [72]:
def min_edit_distance(original_word, all_words):
    distances = defaultdict(list)
    for word in all_words:
        if word != original_word:
            # create dictonary where key represents edit distance
            # from original word, and value is new word, so
            # {1: words one away, 2: words two away}, etc
            distance = edit_distance(word, original_word)
            distances[distance].append(word)
    # find the smallest key, which is the min distance
    min_distance = min(distances.keys())
    return distances[min_distance]


all_words = set()
# unique list of all words
for sent in words_correct:
    for word in sent:
        all_words.add(word)

print(min_edit_distance('minde', all_words))
        
# Test your code as follows
#assert get_candidates("minde") == ['mine', 'mind']

['mine', 'mind']


## Task 4 (15 Marks):

Write a function that takes a (misspelled) sentence and returns the corrected version of that sentence. The system should scan the sentence for words that are not in the dictionary (set of unique words in the training set) and for each word that is not in the dictionary choose a word in the dictionary that has minimal edit distance and has the highest *unigram probability*. 

*Your solution to this should involve `get_candidates`*


In [73]:
def correct(sentence, unigram_model):
    # Write your code here
    result = []

    # for each word
    for word in sentence:
        # if it's in the model already that means it's correct, so append it
        if word in unigram_model:
            result.append(word)

       # if not, get all the words that are close to it by using min_edit_distance
        else:
            correction_candidates = min_edit_distance(word, all_words)


            # The below loop figures out which of the possible substitutes
            # are most common in the dataset, and then appends that substitue
            best_prod, best_word = 0, None
            for c in correction_candidates:
                word_prob = prob(c, unigram_model)
                if word_prob > best_prod:
                    best_prod = word_prob
                    best_word = c
            result.append(best_word)
    return result

print(correct(["this","whitr","cat"], unigram_model))


['this', 'white', 'cat']


## **Task 5** (10 Marks): 
Using the test corpus evaluate the *accuracy* of your method, i.e., how many words from your system's output match the corrected sentence (you should count words that are already spelled correctly and not changed by the system).

In [74]:
def accuracy(test):
    # keep a count of all words that should've been corrected, and how many
    # model got right. Everytime model sees a word that was misspelled - increment 'total'
    # and if it was right - increment 'correct_count'
    correct_count, total = 0.0, 0.0
    for sentence in test:
        original_sentence = sentence['original']
        corrected_sentence = correct(original_sentence, unigram_model)

        true_corrected = sentence['corrected']
        for idx in sentence['indexes']:
            if corrected_sentence[idx] == true_corrected[idx]:
                correct_count += 1
            total += 1
    return correct_count / total

print(accuracy(test))

0.23728813559322035


## **Task 6 (35 Marks):**

Consider a modification to your algorithm that would improve the accuracy of the algorithm developed in Task 3 and 4

* You may resources beyond those provided here.
* You must **not use the test data** in this task.
* Provide a short text describing what you intend to do and why. 
* Full marks for this section may be obtained without an implementation, but an implementation is preferred.
* Your implementation should not consist of more than 50 lines of code

Please note this task is marked according to: demonstration of knowledge from the lecutures (10), originality and appropriateness of solution (10), completeness of description (10) and technical correctness (5)



#### 1. Bigram Model (Implemented) 

The Bigram model is a language model that approximates a probability by using the previous words conditional probability. This type of model of using only the previous word to generate the probability is known as the Markov assumption.  Implementing this modification allowed my language model to gain context when approximating probability, rather than the unigram model which  treats the words as disconnected units via the bag of words perspective. The reason I implemented the Bigram is because it creates a spell checker that prioritizes corrections according to context of the sentence

#### 2. Part of Speech Tagging - Hidden Markov Model (Not Implemented)

Implementing Part-of-Speech tagging into a language model is the process of assigning part-of-speech tags to each word. To implement this into the language model you first need to tokenize the text (already implemented0 and then tag each token. The tags are generated from a tagset, and the tagset I selected while implement Part-of-speech tagging is 45-tag Penn Treebank as it is the most common in English POS tagging. When creating a spell checker, it is useful to implement Parts-of-speech tagging to the words. Understanding what part of speech the word and its neighbours can help the spell checker by giving higher probability to words that grammatical make sense.

A Hidden Markov Model is desirable for part-of-speech tagging tasks as this model calculates the highest probability tag sequence for a given sequence of units (sentences, words, letters, etc.). The goal of HMM tagging is to choose the sequence of tags which is most probable given the observed sequence of words. When juxtaposing HMM and to other part-of-speech tagging techniques HMM is advantageous because it does not take the greedy approach of optimizing words independently.

#### 3. Trigram Model (Not Implemented)

Implementing the Trigram model would have allowed my model to condition on the previous two words rather than the previous word (bigram). This allows for more context when approximating probabilities. However, since my training data set was small it would have been likely that there would’ve been unseen trigrams.

#### 4. Smoothing Technique - Add-one and Backoff (Not Implemented)

Originally, I had implemented Add-one smoothing to my upgraded_bigram model. However, this lowered my overall accuracy score. This made me curious, so I decided to read more into smoothing techniques as I was under the assumption that smoothing was going to improve my spell checkers overall accuracy. I will first discuss what Add-one Smoothing is, and then talk about why Backoff smoothing is a more appropriate smoothing technique for my model. 

Language models assign zero probability to unseen events. Smoothing techniques, such as Add one smoothing, can take care of the zero-probability issue that may arise. Add one smoothing simply adds one to all the bigram counts prior to normalizing them into probabilities. This technique is a non-maximum likelihood estimate because the counts are being altered from their original state in the training data.
    
    • It is important to note that this technique does have cons – such as the fact that it overestimates low probabilities and underestimates frequent probabilities. 


While the add one smoothing technique is useful in solving the issues of zero probabilities, the Back-off model has another feature which allows the context provided by a Trigram model to help approximate the probability. In instances where there are no previous instances of a trigram (Wn-2Wn-1Wn) the Back-off technique then uses the bigram probability. This additional feature allows for context to be used in approximating probability. 



*** (After researching more I ended up taking out add-one smoothing from my model, but wasn't able to add the Backoff Smoothing code due to technical difficulties)

#### 5. Real-word Spelling Errors

In its current state the implemented spellchecker’s accuracy is determined on its ability to correct misspelled words. However, there are instances when a word is technically spelled correctly, but is grammatical incorrect. The first example this can occur is Typographical errors (typo), which is when the correct spelling of the word is known but there is a typo in the word. (typing there instead of three) The second type of Real Word Error not accounted for in the original spell checker is a Cognitive error. These spelling errors occur in instances where the word is spelled incorrectly due to a homophone. (typing peace instead of piece or typing abyss instead of abiss). The justification for implementing Real-word spelling errors comes from a statistic I found while researcher. It states the roughly 25-40% of spelling errors are real words (Kukich 1992).



#### 6. Collecting more data

The collection of more (high quality) data would have increased the overall effectiveness of our spellchecking system. Not only does more data result in more accurate results, but it always increases the size of the model's dictionary.

In [75]:
def build_bigram_model(words_correct):
    
    bigram_counter = defaultdict(int)
    text = []
    # turn list into string
    for i in words_correct:
        text.append(" ".join(i).lower())

    text = " ".join(text)
    text = nltk.word_tokenize(text)


    # '.bigrams' calculates frequency distribution
    bigrams = nltk.bigrams(text)

    for b in bigrams:
        bigram_counter[b] += 1

    return bigram_counter

def correct_upgraded(sentence, unigram_model, bigram_model):
    # Write your code here
    result = []


    for idx, word in enumerate(sentence):
        if word in unigram_model:
            result.append(word)
        else:
            correction_candidates = min_edit_distance(word, all_words)
            best_prod, best_word = 0, None

            # try to use bigram model
            for c in correction_candidates:
                word_prob = bigram_model[(sentence[idx - 1], c)] / (float(unigram_model[c]) + 1)
                if word_prob > best_prod:
                    best_prod = word_prob
                    best_word = c

            # fall back to unigram model
            for c in correction_candidates:
                word_prob = prob(c, unigram_model)
                if word_prob > best_prod:
                    best_prod = word_prob
                    best_word = c

            result.append(best_word)

    return result

bigram_model = build_bigram_model(words_correct)


print(correct_upgraded(["this","whitr","cat"], unigram_model, bigram_model))



['this', 'white', 'cat']


## **Task 7 (5 Marks):**

Repeat the evaluation (as in Task 5) of your new algorithm and show that it outperforms the algorithm from Task 3 and 4

In [76]:
def accuracy_upgraded(test):
    correct_count, total = 0.0, 0.0
    for sentence in test:
        original_sentence = sentence['original']
        corrected_sentence = correct_upgraded(original_sentence, unigram_model, bigram_model)

        true_corrected = sentence['corrected']
        for idx in sentence['indexes']:
            if corrected_sentence[idx] == true_corrected[idx]:
                correct_count += 1
            total += 1
    return correct_count / total


print('Original Method: {}'.format(accuracy(test)))
print('New Method: {}'.format(accuracy_upgraded(test)))


Original Method: 0.23728813559322035
New Method: 0.288135593220339


My original language model performed with 23.73% accuracy. After modification my upgraded language model performed with 28.81% accuracy. 

###### Work Cited
All knowledge from this Natural Language Process report can be attributed to my NLP course as well as the resources listed below:


Chris Manning, Hinrich Schuetze "Foundations of Statistical Natural Language Processing", MIT Press. Cambridge, MA: May 1999. Teaching materials online at: http://nlp.stanford.edu/fsnlp/

Daniel Jurafsky, James H. Martin "Speech and Language Processing - An Introduction to Natural Language Processing, Computational Linguistics and Speech Recognition”, Pearson, Prentice Hall, second edition. ISBN-10: 0131873210. Third Edition draft: https://web.stanford.edu/~jurafsky/slp3/ed3book.pdf

David Sunby "Spelling correction using N-grams"
http://fileadmin.cs.lth.se/cs/education/EDA171/Reports/2009/david.pdf